In [1]:

import os
import pandas as pd
import numpy as np

import time
from scipy import stats
from scipy.interpolate import InterpolatedUnivariateSpline
from sklearn.preprocessing import MinMaxScaler
import anndata
import xgboost

from pyInfinityFlow.InfinityFlow_Utilities import read_fcs_into_anndata
from pyInfinityFlow.InfinityFlow_Utilities import apply_logicle_to_anndata
from pyInfinityFlow.InfinityFlow_Utilities import apply_inverse_logicle_to_anndata
from pyInfinityFlow.InfinityFlow_Utilities import anndata_to_df
from pyInfinityFlow.InfinityFlow_Utilities import write_anndata_to_fcs

os.chdir("/media/kyle_storage/kyle_ferchen/grimes_lab_main/analysis/"\
    "2022_12_07_new_multilin_panel/")

In [2]:
### Helper functions
def pprint(input_list):
    print("\n".join(input_list))


def scale_feature(input_array, min_threshold_percentile, max_threshold_percentile):
    input_array = np.array(list(input_array))
    min_threshold = np.percentile(input_array, min_threshold_percentile)
    max_threshold = np.percentile(input_array, max_threshold_percentile)
    input_array[input_array < min_threshold] = min_threshold
    input_array[input_array > max_threshold] = max_threshold
    return(MinMaxScaler().fit(input_array.reshape(-1,1)).transform(input_array.reshape(-1,1)).reshape(-1,))


def make_spline_function_for_reference(reference_signal, n=100):
    start_time = time.time()
    # n = 100    # Number of points to consider during integration of KDE model
    # Build the KDE model
    reference_kde_model = stats.gaussian_kde(reference_signal)
    tmp_total_area = reference_kde_model.integrate_box_1d(0, 1)
    tmp_percentiles = np.array([reference_kde_model.integrate_box(0, tmp_point) / tmp_total_area for tmp_point in np.linspace(0,1,n)])
    # Remove values out of range
    flag_out_of_range = (tmp_percentiles <= 0) | (tmp_percentiles >= 1)
    x_spline = tmp_percentiles[~flag_out_of_range]
    y_spline = np.linspace(0,1,n)[~flag_out_of_range]
    # Remove duplicates
    flag_duplicates = np.unique(x_spline, return_index=True)[-1]
    x_spline = x_spline[flag_duplicates]
    y_spline = y_spline[flag_duplicates]
    # Add boundaries
    x_spline = np.concatenate(([0], x_spline, [1]))
    y_spline = np.concatenate(([0], y_spline, [1]))
    reference_spline_function = InterpolatedUnivariateSpline(x_spline, y_spline, k=1, bbox=[0,1], ext=3)
    end_time = time.time()
    print("\tSpline Function Built after {:.2f} seconds.".format(end_time - start_time))
    return(reference_spline_function)


def map_input_to_reference_spline(input_signal, reference_spline_function):
    input_ranks = pd.Series(np.argsort(input_signal), index=list(range(len(input_signal)))).sort_values().index.values
    return(np.array([float(reference_spline_function(item)) for item in ((input_ranks + 1) / (len(input_signal)+2))]))

In [8]:
### Read fcs files
new_flow = read_fcs_into_anndata(\
    "output/new_multilin_panel_with_umap_and_leiden.fcs")

old_infinity = read_fcs_into_anndata(\
    "/media/kyle_storage/kyle_ferchen/grimes_lab_main/analysis/"\
    "2022_10_26_pyInfinityFlow_grimes_data/outputs/"\
    "merged_regression_results/merged_flow.fcs")

In [9]:
### Read input for old infinityflow to get CD131 signal
cd131_backbone = read_fcs_into_anndata("input/old_infinityflow/"\
    "export_KF_Infinity_Flow_Add_PEs-Infinity_Markers_PE-CD131_Unmixed_Live.fcs")

In [10]:
print(cd131_backbone.var.to_string())

                            name  USE_LOGICLE  LOGICLE_T  LOGICLE_W  LOGICLE_M  LOGICLE_A  LOGICLE_APPLIED  IMPUTED
FSC-A                                   False  3000000.0        0.0        3.0        1.0            False    False
FSC-H                                   False  3000000.0        0.0        3.0        1.0            False    False
FSC-W                                   False  3000000.0        0.0        3.0        1.0            False    False
SSC-A                                   False  3000000.0        0.0        3.0        1.0            False    False
SSC-B-A                                 False  3000000.0        0.0        3.0        1.0            False    False
SSC-B-H                                 False  3000000.0        0.0        3.0        1.0            False    False
SSC-B-W                                 False  3000000.0        0.0        3.0        1.0            False    False
SSC-H                                   False  3000000.0        0.0     

In [11]:
old_infinity_markers = old_infinity.var.loc[\
    old_infinity.var["name"].str.contains('InfinityMarker')].index.values

In [12]:
old_with_cd131 = anndata.AnnData(\
    X=np.concatenate([\
            cd131_backbone.X,
            old_infinity[:,old_infinity_markers].X.toarray()],
        axis=1),
    obs=cd131_backbone.obs,
    var=pd.concat([\
            cd131_backbone.var, 
            old_infinity.var.loc[old_infinity_markers]], 
        axis=0))

In [13]:
old_with_cd131.var.loc[["SSC-B-A", "SSC-W", "AF-A"], "USE_LOGICLE"] = False
old_with_cd131.var.loc["PE-A", "name"] = "CD131"

In [14]:
### Apply logicle function to normalize the data
apply_logicle_to_anndata(old_with_cd131)
apply_logicle_to_anndata(new_flow)

In [35]:
# Fix range from previous InfinityFlow results to be 0-1
feaures_fix_range = old_with_cd131.var.loc[\
    old_with_cd131.var["USE_LOGICLE"]].index.values

old_with_cd131[:,feaures_fix_range].X = np.where(\
    old_with_cd131[:,feaures_fix_range].X > 1,
    1,
    old_with_cd131[:,feaures_fix_range].X)

old_with_cd131[:,feaures_fix_range].X = np.where(\
    old_with_cd131[:,feaures_fix_range].X < 0,
    0,
    old_with_cd131[:,feaures_fix_range].X)

In [37]:
map_old_to_new = pd.Series({\
    'APC-A': 'CD371',
    'APC-Cy7-A': 'CD48',
    'AlexaFluor647-A': 'AlexaFluor647-A',
    'AlexaFluor700-A': 'BUV805-A',
    'BUV395-A': 'BUV395-A',
    'BUV615-A': 'INTEGRIN_B7',
    'BUV737-A': 'BUV737-A',
    'BUV805-A': 'CD105',
    'BV421-A': 'PE-A',
    'BV480-A': 'BV480-A',
    'BV510-A': 'V450-A',
    'BV570-A': 'BV570-A',
    'BV605-A': 'CD41',
    'BV650-A': 'BV650-A',
    'BV711-A': 'BV711-A',
    'BV750-A': 'BV750-A',
    'BV785-A': 'CD127',
    'GFP-A': 'GFP-A',
    'PE-A': 'CX3CR1',
    'PE-Cy5-A': 'PE-Cy5-A',
    'PE-Cy5.5-A': 'PE-Cy7-A',
    'PE-Cy7-A': 'CD125',
    'PacificBlue-A': 'FCER1A',
    'PerCP-Cy5.5-A': 'APC-A',
    'PerCP-eFluor710-A': 'CD201'})

In [38]:
### Sample the old data to 2E6 cells
sampled_cells = old_with_cd131.obs.sample(int(2e6)).sort_values(\
    by="cell_number").index.values
old_sampled = old_with_cd131[sampled_cells,:]

In [39]:
### Sample the new data to create distribution maps for old data
sampled_new_cells = new_flow.obs.sample(int(2e5)).sort_values(\
    by="cell_number").index.values
new_sampled = new_flow[sampled_new_cells,:]

In [40]:
### Build reference functions to map distributions
ref_functions = anndata_to_df(new_sampled)[map_old_to_new.index.values].apply(\
    lambda x: make_spline_function_for_reference(x.values), axis=0)

	Spline Function Built after 2.93 seconds.
	Spline Function Built after 3.25 seconds.
	Spline Function Built after 3.09 seconds.
	Spline Function Built after 3.63 seconds.
	Spline Function Built after 2.60 seconds.
	Spline Function Built after 2.08 seconds.
	Spline Function Built after 3.52 seconds.
	Spline Function Built after 3.62 seconds.
	Spline Function Built after 2.00 seconds.
	Spline Function Built after 3.56 seconds.
	Spline Function Built after 3.55 seconds.
	Spline Function Built after 3.35 seconds.
	Spline Function Built after 1.91 seconds.
	Spline Function Built after 2.78 seconds.
	Spline Function Built after 2.13 seconds.
	Spline Function Built after 3.54 seconds.
	Spline Function Built after 2.40 seconds.
	Spline Function Built after 3.63 seconds.
	Spline Function Built after 2.98 seconds.
	Spline Function Built after 2.40 seconds.
	Spline Function Built after 3.56 seconds.
	Spline Function Built after 1.91 seconds.
	Spline Function Built after 1.92 seconds.
	Spline Fun

In [41]:
ref_functions

APC-A                <scipy.interpolate._fitpack2.InterpolatedUniva...
APC-Cy7-A            <scipy.interpolate._fitpack2.InterpolatedUniva...
AlexaFluor647-A      <scipy.interpolate._fitpack2.InterpolatedUniva...
AlexaFluor700-A      <scipy.interpolate._fitpack2.InterpolatedUniva...
BUV395-A             <scipy.interpolate._fitpack2.InterpolatedUniva...
BUV615-A             <scipy.interpolate._fitpack2.InterpolatedUniva...
BUV737-A             <scipy.interpolate._fitpack2.InterpolatedUniva...
BUV805-A             <scipy.interpolate._fitpack2.InterpolatedUniva...
BV421-A              <scipy.interpolate._fitpack2.InterpolatedUniva...
BV480-A              <scipy.interpolate._fitpack2.InterpolatedUniva...
BV510-A              <scipy.interpolate._fitpack2.InterpolatedUniva...
BV570-A              <scipy.interpolate._fitpack2.InterpolatedUniva...
BV605-A              <scipy.interpolate._fitpack2.InterpolatedUniva...
BV650-A              <scipy.interpolate._fitpack2.InterpolatedUniva...
BV711-

In [42]:
mapped_old_sampled = anndata_to_df(old_sampled)[map_old_to_new.values].rename(\
    pd.Series(map_old_to_new.index.values, 
        index=map_old_to_new.values).to_dict(), axis=1).apply(\
            lambda x: map_input_to_reference_spline(x.values, 
                ref_functions[x.name]), axis=0)


In [43]:
infinity_markers_to_impute = np.setdiff1d(\
    old_sampled.var.loc[\
        old_sampled.var["name"].str.contains("InfinityMarker")].index.values,
    map_old_to_new.values)

In [44]:
mapped_old_sampled

,APC-A,APC-Cy7-A,AlexaFluor647-A,AlexaFluor700-A,BUV395-A,BUV615-A,BUV737-A,BUV805-A,BV421-A,BV480-A,...,BV750-A,BV785-A,GFP-A,PE-A,PE-Cy5-A,PE-Cy5.5-A,PE-Cy7-A,PacificBlue-A,PerCP-Cy5.5-A,PerCP-eFluor710-A
:2,0.424410,0.427024,0.318727,0.629068,0.385917,0.258708,0.225279,0.262861,0.267747,0.282168,...,0.470559,0.294988,0.260375,0.316098,0.245570,0.383413,0.261011,0.258915,0.295650,0.258741
:9,0.298738,0.593379,0.547007,0.216214,0.255422,0.277736,0.288845,0.574605,0.263170,0.246862,...,0.257303,0.313108,0.277859,0.255394,0.205901,0.245999,0.268969,0.249402,0.276304,0.248763
:12,0.360345,0.760444,0.288602,0.251247,0.366382,0.256931,0.572789,0.312039,0.257457,0.349203,...,0.272559,0.249309,0.625160,0.262300,0.268274,0.342825,0.249980,0.238896,0.269036,0.227285
:15,0.373744,0.310805,0.372770,0.654271,0.337369,0.267897,0.268465,0.270333,0.261384,0.276535,...,0.465346,0.303536,0.244474,0.246691,0.249738,0.264059,0.261641,0.237655,0.246307,0.249793
:21,0.528509,0.436790,0.272538,0.545539,0.381612,0.248827,0.225760,0.244935,0.304078,0.254993,...,0.540195,0.295832,0.259474,0.257671,0.272810,0.303141,0.255751,0.245275,0.284053,0.254921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
:5751988,0.421799,0.323730,0.276350,0.634028,0.320036,0.245280,0.248830,0.255514,0.258414,0.283067,...,0.539166,0.250922,0.225134,0.224334,0.235807,0.270046,0.254396,0.228469,0.251535,0.226648
:5751989,0.405053,0.345010,0.298547,0.627195,0.340545,0.259088,0.241085,0.253006,0.280349,0.258820,...,0.507871,0.280189,0.223467,0.233640,0.249948,0.266424,0.257775,0.232999,0.245839,0.248080
:5751995,0.254168,0.305231,0.403707,0.275900,0.250421,0.263761,0.256414,0.401508,0.259698,0.242973,...,0.259219,0.315329,0.258384,0.248410,0.258624,0.275813,0.267991,0.244782,0.225412,0.259059
:5752000,0.543972,0.642238,0.266234,0.504844,0.461112,0.256939,0.340136,0.270507,0.275760,0.454349,...,0.553453,0.270768,0.276275,0.304205,0.262265,0.264044,0.255772,0.260270,0.388499,0.278470


In [45]:
### Build regression models to predict infinity markers on new panel
rmodels_old_infinity = {}
for tmp_marker in infinity_markers_to_impute:
    print("Working on {}...".format(tmp_marker))
    tmp_model = xgboost.XGBRegressor(n_jobs=12)
    tmp_model.fit(mapped_old_sampled,
                old_sampled[:,tmp_marker].X.toarray().reshape(-1))
    rmodels_old_infinity[tmp_marker] = tmp_model


Working on Bhlhe40-GFP...
Working on CD102...
Working on CD107a...
Working on CD115...
Working on CD11a...
Working on CD11c...
Working on CD123...
Working on CD124...
Working on CD126...
Working on CD129...
Working on CD137...
Working on CD14...
Working on CD140a...
Working on CD155...
Working on CD159a...
Working on CD160...
Working on CD163...
Working on CD169...
Working on CD170...
Working on CD172a...
Working on CD182...
Working on CD183...
Working on CD193...
Working on CD1d...
Working on CD2...
Working on CD20...
Working on CD200...
Working on CD200R3...
Working on CD200r...
Working on CD204...
Working on CD205...
Working on CD213a1...
Working on CD21_35...
Working on CD22...
Working on CD223...
Working on CD23...
Working on CD24...
Working on CD25...
Working on CD26...
Working on CD270...
Working on CD274...
Working on CD278...
Working on CD279...
Working on CD29...
Working on CD300LG...
Working on CD301b...
Working on CD317...
Working on CD357...
Working on CD36...
Working on C

In [46]:
pd.Series(map_old_to_new.index.values, 
        index=map_old_to_new.values).to_dict()

{'CD371': 'APC-A',
 'CD48': 'APC-Cy7-A',
 'AlexaFluor647-A': 'AlexaFluor647-A',
 'BUV805-A': 'AlexaFluor700-A',
 'BUV395-A': 'BUV395-A',
 'INTEGRIN_B7': 'BUV615-A',
 'BUV737-A': 'BUV737-A',
 'CD105': 'BUV805-A',
 'PE-A': 'BV421-A',
 'BV480-A': 'BV480-A',
 'V450-A': 'BV510-A',
 'BV570-A': 'BV570-A',
 'CD41': 'BV605-A',
 'BV650-A': 'BV650-A',
 'BV711-A': 'BV711-A',
 'BV750-A': 'BV750-A',
 'CD127': 'BV785-A',
 'GFP-A': 'GFP-A',
 'CX3CR1': 'PE-A',
 'PE-Cy5-A': 'PE-Cy5-A',
 'PE-Cy7-A': 'PE-Cy5.5-A',
 'CD125': 'PE-Cy7-A',
 'FCER1A': 'PacificBlue-A',
 'APC-A': 'PerCP-Cy5.5-A',
 'CD201': 'PerCP-eFluor710-A'}

In [47]:
old_sampled.var.loc[old_sampled.var["name"].str.contains("InfinityMarker")].index.values

array(['Bhlhe40-GFP', 'CD102', 'CD105', 'CD107a', 'CD115', 'CD11a',
       'CD11c', 'CD123', 'CD124', 'CD125', 'CD126', 'CD127', 'CD129',
       'CD137', 'CD14', 'CD140a', 'CD155', 'CD159a', 'CD160', 'CD163',
       'CD169', 'CD170', 'CD172a', 'CD182', 'CD183', 'CD193', 'CD1d',
       'CD2', 'CD20', 'CD200', 'CD200R3', 'CD200r', 'CD201', 'CD204',
       'CD205', 'CD213a1', 'CD21_35', 'CD22', 'CD223', 'CD23', 'CD24',
       'CD25', 'CD26', 'CD270', 'CD274', 'CD278', 'CD279', 'CD29',
       'CD300LG', 'CD301b', 'CD317', 'CD357', 'CD36', 'CD366', 'CD371',
       'CD38', 'CD41', 'CD43', 'CD44', 'CD48', 'CD49a', 'CD49b', 'CD49d',
       'CD51', 'CD54', 'CD61', 'CD63', 'CD64', 'CD69', 'CD73', 'CD79b',
       'CD81', 'CD84', 'CD85k', 'CD90_2', 'CD95', 'CX3CR1', 'CXCR4',
       'Csf1r-GFP', 'ESAM', 'Epx-cre-tdTomato', 'F4_80', 'FCER1A',
       'Fucci-hCdt1-mCherry', 'Fucci-hGem-Venus', 'Gata1-mCherry',
       'Gata2-Venus', 'Gfi1-tdTomato', 'IL33r', 'INTEGRIN_B7', 'I_A_I_E',
       'IgD', 'IgM

In [48]:
old_backbone_to_add = ["APC-Cy7-A", "AlexaFluor700-A", "BUV496-A", "BUV661-A", "BV421-A", "BV510-A", "BV605-A"]

In [50]:
### Build regression models to predict infinity markers on new panel
rmodels_old_backbone = {}
for tmp_marker in old_backbone_to_add:
    print("Working on {}...".format(tmp_marker))
    tmp_model = xgboost.XGBRegressor(n_jobs=12)
    tmp_model.fit(mapped_old_sampled,
                old_sampled[:,tmp_marker].X.toarray().reshape(-1))
    rmodels_old_backbone[tmp_marker] = tmp_model

Working on APC-Cy7-A...
Working on AlexaFluor700-A...
Working on BUV496-A...
Working on BUV661-A...
Working on BV421-A...
Working on BV510-A...
Working on BV605-A...


In [52]:
### Make predictions (previous infinity markers)
preds_infinity_markers = {}
for tmp_marker in rmodels_old_infinity:
    print("Working on {}...".format(tmp_marker))
    preds_infinity_markers[tmp_marker] = rmodels_old_infinity[\
        tmp_marker].predict(\
            new_flow[:,mapped_old_sampled.columns.values].X.toarray())


Working on Bhlhe40-GFP...
Working on CD102...
Working on CD107a...
Working on CD115...
Working on CD11a...
Working on CD11c...
Working on CD123...
Working on CD124...
Working on CD126...
Working on CD129...
Working on CD137...
Working on CD14...
Working on CD140a...
Working on CD155...
Working on CD159a...
Working on CD160...
Working on CD163...
Working on CD169...
Working on CD170...
Working on CD172a...
Working on CD182...
Working on CD183...
Working on CD193...
Working on CD1d...
Working on CD2...
Working on CD20...
Working on CD200...
Working on CD200R3...
Working on CD200r...
Working on CD204...
Working on CD205...
Working on CD213a1...
Working on CD21_35...
Working on CD22...
Working on CD223...
Working on CD23...
Working on CD24...
Working on CD25...
Working on CD26...
Working on CD270...
Working on CD274...
Working on CD278...
Working on CD279...
Working on CD29...
Working on CD300LG...
Working on CD301b...
Working on CD317...
Working on CD357...
Working on CD36...
Working on C

In [53]:
preds_old_backbone = {}
for tmp_marker in rmodels_old_backbone:
    print("Working on {}...".format(tmp_marker))
    preds_old_backbone[tmp_marker] = rmodels_old_backbone[\
        tmp_marker].predict(\
            new_flow[:,mapped_old_sampled.columns.values].X.toarray())

Working on APC-Cy7-A...
Working on AlexaFluor700-A...
Working on BUV496-A...
Working on BUV661-A...
Working on BV421-A...
Working on BV510-A...
Working on BV605-A...


In [54]:
# Map target names to old backbone antibodies
preds_old_backbone = pd.DataFrame(preds_old_backbone).rename(\
        old_with_cd131.var.loc[old_backbone_to_add, "name"].to_dict(),
    axis=1)

preds_old_backbone.head()

,CD49f,CD86,CD9,CD71,CD34,CD62L,CD31
0,0.459577,0.259098,0.435904,0.340476,0.294711,0.334237,0.269808
1,0.583818,0.259554,0.338676,0.337604,0.449510,0.296884,0.556567
2,0.453581,0.252547,0.403327,0.458152,0.304906,0.315412,0.280435
3,0.333683,0.240525,0.291171,0.261209,0.258951,0.262318,0.287023
4,0.541212,0.247726,0.374919,0.316505,0.376912,0.327803,0.587470


In [55]:
to_add = pd.concat([\
        pd.DataFrame(preds_infinity_markers),
        preds_old_backbone],
    axis=1)

to_add.head()

,Bhlhe40-GFP,CD102,CD107a,CD115,CD11a,CD11c,CD123,CD124,CD126,CD129,...,Tox-tdTomato,VISTA,Wnt_GFP,CD49f,CD86,CD9,CD71,CD34,CD62L,CD31
0,0.483504,0.282551,0.370626,0.251141,0.781720,0.260200,0.240645,0.244336,0.278635,0.246018,...,0.226551,0.668736,0.203759,0.459577,0.259098,0.435904,0.340476,0.294711,0.334237,0.269808
1,0.257846,0.316142,0.460162,0.249720,0.655819,0.280438,0.277563,0.238089,0.317050,0.234272,...,0.244293,0.471549,0.247632,0.583818,0.259554,0.338676,0.337604,0.449510,0.296884,0.556567
2,0.551759,0.285235,0.504323,0.268991,0.749644,0.245721,0.241985,0.250730,0.268375,0.248457,...,0.225282,0.696585,0.238999,0.453581,0.252547,0.403327,0.458152,0.304906,0.315412,0.280435
3,0.237291,0.266469,0.371250,0.260524,0.552190,0.249978,0.243887,0.239769,0.251683,0.238476,...,0.240721,0.419808,0.233979,0.333683,0.240525,0.291171,0.261209,0.258951,0.262318,0.287023
4,0.252854,0.307304,0.435956,0.258098,0.671062,0.273384,0.254588,0.238517,0.289072,0.236819,...,0.249280,0.469187,0.251508,0.541212,0.247726,0.374919,0.316505,0.376912,0.327803,0.587470


In [56]:
to_add_var = pd.DataFrame({\
        "name": ["Imputed_" + item for item in to_add.columns.values],
        "USE_LOGICLE": True,
        "LOGICLE_T":3000000.0,
        "LOGICLE_W": 0.0,
        "LOGICLE_M": 3.0,
        "LOGICLE_A": 1.0,
        "LOGICLE_APPLIED": True,
        "IMPUTED": True},
    index=to_add.columns.values)
to_add_var.head()

,name,USE_LOGICLE,LOGICLE_T,LOGICLE_W,LOGICLE_M,LOGICLE_A,LOGICLE_APPLIED,IMPUTED
Bhlhe40-GFP,Imputed_Bhlhe40-GFP,True,3000000.0,0.0,3.0,1.0,True,True
CD102,Imputed_CD102,True,3000000.0,0.0,3.0,1.0,True,True
CD107a,Imputed_CD107a,True,3000000.0,0.0,3.0,1.0,True,True
CD115,Imputed_CD115,True,3000000.0,0.0,3.0,1.0,True,True
CD11a,Imputed_CD11a,True,3000000.0,0.0,3.0,1.0,True,True


In [57]:
final = anndata.AnnData(\
    X=np.concatenate([\
            new_flow.X,
            to_add.values],
        axis=1),
    obs=new_flow.obs,
    var=pd.concat([\
            new_flow.var, 
            to_add_var], 
        axis=0))

In [58]:
final.write("output/infinity_flow/"\
    "infinity_flow_reprediction_on_new_panal_logicle_redo.h5ad")

In [59]:
from pyInfinityFlow.InfinityFlow_Utilities import apply_inverse_logicle_to_anndata
apply_inverse_logicle_to_anndata(final)

()

In [60]:
# final.X[final.X > 1e10] = 1e10

In [61]:
from pyInfinityFlow.InfinityFlow_Utilities import write_anndata_to_fcs

write_anndata_to_fcs(\
    final, 
    "output/infinity_flow/infinity_flow_reprediction_on_new_panal_logicle_redo.fcs",
    add_umap=False,
    verbosity=3)

Omitting spillover matrix...
WARNING! TEXT segment value for key $P26S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P27S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P28S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P29S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P30S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P31S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P32S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P33S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P34S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P35S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P36S is empty. Excluding from written file.
WARNING! TEXT segment value for key $P38S is empty. Excluding from written file.